<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/test_notebooks/msc_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasyfikacja Toksycznych Komentarzy z DistilBERT i Analiza XAI

Ten notebook demonstruje:
1. Fine-tuning modelu DistilBERT dla wieloklasowej klasyfikacji toksyczności
2. Metody wyjaśnialności (Integrated Gradients, Input×Gradient)
3. Analiza warstwowa metodą sondowania liniowego
4. Inżynieria reprezentacji do sterowania zachowaniem modelu


## 1. Konfiguracja Środowiska i Zależności


In [ ]:
!uv pip install transformers datasets captum quantus accelerate

In [ ]:
from google.colab import drive
drive.mount('/drive')

## 2. Wczytywanie i Eksploracja Danych


In [ ]:
import pandas as pd

# Ścieżka do pliku CSV z danymi Jigsaw Toxic Comment
csv_path = '/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv'

try:
    dataframe = pd.read_csv(csv_path)
    print("Plik CSV wczytany pomyślnie!")
    display(dataframe.head())
except FileNotFoundError:
    print(f"Błąd: Nie znaleziono pliku pod ścieżką {csv_path}")
except Exception as e:
    print(f"Wystąpił błąd: {e}")


## 3. Preprocessing i Czyszczenie Danych


In [ ]:
import re

def clean_text(example):
    """
    Stosuje kompleksowe operacje czyszczenia na polu 'comment_text'.
    
    Funkcja jest kluczowa dla zapewnienia spójności danych między treningiem a ewaluacją.
    Usuwa szum i normalizuje format tekstów komentarzy.
    
    Argumenty:
        example: Słownik zawierający klucz 'comment_text' z tekstem do oczyszczenia
        
    Zwraca:
        Zaktualizowany słownik example z oczyszczonym tekstem
        
    Operacje czyszczenia:
        - Konwersja na małe litery (wymagane dla modeli BERT typu uncased)
        - Usunięcie URL (wzorce http/https/www)
        - Usunięcie adresów IP (np. 192.168.1.1)
        - Usunięcie metadanych Wikipedii (talk pages, timestampy UTC)
        - Usunięcie znaków specjalnych i normalizacja białych znaków
    """
    text = example['comment_text']
    
    # Konwersja na małe litery dla modeli uncased BERT
    text = text.lower()
    
    # Usunięcie URL (wzorce http/https i www)
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Usunięcie adresów IP (np. 192.168.1.1)
    text = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', '', text)
    
    # Usunięcie metadanych specyficznych dla Wikipedii
    text = re.sub(r'\(talk\)', '', text)
    text = re.sub(r'\d{2}:\d{2}, \w+ \d{1,2}, \d{4} \(utc\)', '', text)
    
    # Usunięcie znaków nowej linii i spacji niepodzielnych
    text = text.replace('\n', ' ')
    text = text.replace('\xa0', ' ')
    
    # Usunięcie cudzysłowów z początku/końca
    text = text.strip(' "')
    
    # Normalizacja białych znaków (zamiana wielokrotnych spacji na jedną)

    text = re.sub(r'\s+', ' ', text).strip()    return example

        example['comment_text'] = text

In [ ]:
import datasets

# Ograniczenie do pierwszych 2000 próbek dla szybszego eksperymentowania
# Wartość 2000 zapewnia wystarczającą różnorodność danych przy krótkim czasie przetwarzania
train_dataframe = dataframe.head(2000)
dataset = datasets.Dataset.from_pandas(train_dataframe)

In [ ]:
print("Czyszczenie danych...")
cleaned_dataset = dataset.map(clean_text)
print("Czyszczenie danych zakończone!")

In [ ]:
# Porównanie danych przed i po czyszczeniu
print("=== PRZED CZYSZCZENIEM ===")
print(dataset[1]['comment_text'])
print("\n" + dataset[6]['comment_text'])
print("\n" + dataset[0]['comment_text'])

print("\n\n=== PO CZYSZCZENIU ===")
print(cleaned_dataset[1]['comment_text'])
print("\n" + cleaned_dataset[6]['comment_text'])
print("\n" + cleaned_dataset[0]['comment_text'])

## 4. Tokenizacja

In [ ]:
from transformers import AutoTokenizer

# Załadowanie tokenizera dla modelu DistilBERT uncased
model_checkpoint = "distilbert-base-uncased"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    print("Tokenizer załadowany pomyślnie!")
except Exception as e:
    print(f"Błąd ładowania tokenizera: {e}")

In [ ]:
def tokenize_function(examples):
    """
    Tokenizuje batch tekstów dla modeli BERT.
    
    Parametry tokenizacji zoptymalizowane dla DistilBERT i klasyfikacji komentarzy.
    
    Argumenty:
        examples: Batch przykładów zawierających pole 'comment_text'
        
    Zwraca:
        Słownik z kluczami: input_ids (ID tokenów), attention_mask (maska paddingu)
        
    Parametry:
        - padding="max_length": Wyrównuje krótkie komentarze do jednolitej długości
        - truncation=True: Obcina komentarze przekraczające maksymalną długość
        - max_length=256: Równowaga między kontekstem a szybkością
                         (DistilBERT obsługuje do 512, ale 256 wystarczy dla komentarzy)
    
    Uwagi:
        256 tokenów to ~150-200 słów, co pokrywa większość komentarzy w zbiorze Jigsaw.
        Większa wartość zwiększa zużycie pamięci GPU bez znaczącej poprawy jakości.
    """
    return tokenizer(
        examples["comment_text"],
        padding="max_length",

        truncation=True,print("Tokenizacja zakończona!")

        max_length=256  # Magic number: 256 tokenów = optymalna długość dla komentarzytokenized_dataset = cleaned_dataset.map(tokenize_function, batched=True)

    )print("Tokenizacja zbioru danych...")

# Zastosowanie tokenizacji z batchingiem dla wydajności

In [ ]:
# Przykład zetokenizowanego wpisu
print("=== Przykład Zetokenizowanego Wpisu ===")
print(tokenized_dataset[0])

## 5. Przygotowanie Etykiet dla Klasyfikacji Multi-Label

In [ ]:
import numpy as np

# Definicja 6 kolumn etykiet toksyczności w kolejności
label_columns = [
    'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'
]

def create_labels_column(example):
    """
    Konsoliduje 6 osobnych etykiet binarnych w jedną tablicę 'labels'.
    
    Przekształca format danych z wielu kolumn binarnych (0/1) na jedną listę,
    która jest wymagana przez modele PyTorch dla klasyfikacji multi-label.
    
    Argumenty:
        example: Słownik zawierający pojedyncze kolumny etykiet
        
    Zwraca:
        Zaktualizowany example z dodatkową listą 'labels'
        
    Format wyjściowy:
        example['labels'] = [toxic, severe_toxic, obscene, threat, insult, identity_hate]
        Każda wartość to float (0.0 lub 1.0) dla kompatybilności z PyTorch
        
    Uwagi:
        Konwersja na float jest kluczowa - PyTorch wymaga float32 dla BCEWithLogitsLoss
    """
    labels_list = [float(example[col]) for col in label_columns]
    example['labels'] = labels_list

    return exampleprint(final_dataset[6])

print("\n=== Przykładowy Przetworzony Wpis ===")

print("Konsolidacja etykiet...")# Wyświetlenie przykładowego przetworzono wpisu

final_dataset = tokenized_dataset.map(create_labels_column)
print("Etykiety skonsolidowane!")

In [ ]:
# Usunięcie niepotrzebnych kolumn w celu przygotowania do treningu
columns_to_remove = [
    'id', 'comment_text', 'toxic', 'severe_toxic',
    'obscene', 'threat', 'insult', 'identity_hate'
]

print(f"Oryginalne kolumny: {final_dataset.column_names}")
final_dataset = final_dataset.remove_columns(columns_to_remove)
print(f"Pozostałe kolumny: {final_dataset.column_names}")

# Ustawienie formatu zbioru danych na tensory PyTorch
try:
    final_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
    print("\nFormat zbioru danych ustawiony na 'torch'!")
except ImportError:
    print("\nPyTorch nie jest zainstalowany. Pominięto .set_format('torch').")
    print("Zainstaluj komendą: pip install torch")

print("\n=== Ostateczny Element Gotowy dla Modelu ===")
print(final_dataset[6])

## 6. Konfiguracja i Trening Modelu

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6  # Sześć kategorii toksyczności (toxic, severe_toxic, obscene, threat, insult, identity_hate)

# Załadowanie wstępnie wytrenowanego modelu i konfiguracja dla klasyfikacji multi-label
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    problem_type="multi_label_classification"  # Multi-label: komentarz może mieć wiele etykiet jednocześnie
)

print("Model załadowany pomyślnie!")
print("Skonfigurowany dla klasyfikacji multi-label z 6 wyjściami.")

In [ ]:
# Podział zbioru danych na zestawy treningowy i ewaluacyjny
data_splits = final_dataset.train_test_split(test_size=0.2, seed=42)  # 20% na ewaluację, 80% na trening

train_dataset = data_splits['train']
evaluation_dataset = data_splits['test']

print("Podział danych zakończony:")
print(f"  Próbki treningowe: {len(train_dataset)}")
print(f"  Próbki ewaluacyjne: {len(evaluation_dataset)}")

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

def compute_metrics(prediction: EvalPrediction):
    """
    Oblicza metryki ewaluacyjne dla klasyfikacji multi-label.
    
    Funkcja konwertuje logity na prawdopodobieństwa (sigmoid), a następnie
    binaryzuje przez próg 0.5 i oblicza kluczowe metryki.
    
    Argumenty:
        prediction: Obiekt EvalPrediction z polami predictions i label_ids
        
    Zwraca:
        Słownik z metrykami: f1_micro, accuracy
        
    Metryki:
        - f1_micro: Mikro-uśredniony F1 (dobre dla niezbalansowanych etykiet;
                   traktuje każdą próbkę-etykietę jako oddzielny przypadek)
        - accuracy: Ogólna dokładność (procent poprawnie sklasyfikowanych elementów)
        
    Próg klasyfikacji (threshold = 0.5):
        Wartość 0.5 jest standardem dla klasyfikacji binarnej.
        Prawdopodobieństwo > 0.5 => etykieta pozytywna (1)
        Prawdopodobieństwo <= 0.5 => etykieta negatywna (0)
        Można dostosować próg dla różnej równowagi precyzja/recall.
    """
    # Zastosowanie sigmoidu do logitów w celu uzyskania prawdopodobieństw
    logits = prediction.predictions
    probabilities = 1 / (1 + np.exp(-logits))  # Sigmoid: logits -> [0, 1]
    
    # Konwersja prawdopodobieństw na predykcje binarne (próg = 0.5)
    threshold = 0.5  # Magic number: standardowy próg decyzyjny dla klasyfikacji binarnej

    predictions = (probabilities > threshold).astype(int)    }

            'accuracy': overall_accuracy

    labels = prediction.label_ids        'f1_micro': f1_micro,

        return {

    # Mikro-uśredniony F1 (dobry dla niezbalansowanych etykiet)    

    f1_micro = f1_score(labels, predictions, average='micro')    overall_accuracy = accuracy_score(labels.flatten(), predictions.flatten())

        # Ogólna dokładność dla wszystkich etykiet

In [ ]:
from transformers import TrainingArguments

model_output_dir = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-finetuned"

training_args = TrainingArguments(
    output_dir=model_output_dir,
    learning_rate=2e-5,  # 2e-5 to standardowy LR dla fine-tuningu BERT (zbyt wysoki destabilizuje trening)
    num_train_epochs=3,  # 3 epoki to standard dla BERT (więcej = ryzyko overfittingu)
    per_device_train_batch_size=8,  # Batch size 8 bezpieczny dla GPU T4 w Colab (zwiększ do 16-32 dla silniejszych GPU)
    per_device_eval_batch_size=8,
    weight_decay=0.01,  # Regularyzacja L2 zapobiegająca overfittingowi
    eval_strategy="epoch",  # Ewaluacja po każdej epoce (monitorowanie postępów)
    save_strategy="epoch",  # Zapis checkpointu po każdej epoce
    logging_steps=5,  # Logowanie co 5 kroków treningowych
    load_best_model_at_end=True,  # Wczytanie najlepszego modelu po zakończeniu (według f1_micro)
    metric_for_best_model="f1_micro",  # Kryterium wyboru najlepszego modelu
    report_to="none",  # Wyłączenie integracji z wandb
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=evaluation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("=== Rozpoczęcie Treningu ===")
trainer.train()
print("=== Trening Zakończony ===")


In [ ]:
import os
from datetime import datetime

# Utworzenie katalogu zapisu z timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
base_path = "/drive/MyDrive/msc-project/models/final_distilbert_jigsaw"
save_directory = f"{base_path}_{timestamp}"

# Zapis zarówno modelu jak i tokenizera
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model i tokenizer zapisane w: {save_directory}")

## 7. Wnioskowanie i Testowanie Modelu

In [ ]:
import torch
import torch.nn.functional as F

# Przełączenie modelu w tryb ewaluacji
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Testowe zdanie z mieszaną treścią toksyczną i pozytywną
test_text = "you are a fucking moron, who should die in hell but I love your lovely kitten"

# Tokenizacja wejścia
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True).to(device)

# Uruchomienie wnioskowania
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    # Użycie sigmoid dla klasyfikacji multi-label (każda etykieta niezależnie)
    probabilities = torch.sigmoid(logits)

# Wyświetlenie wyników dla każdej etykiety
labels_list = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(f"Tekst: '{test_text}'\n")
print("Prawdopodobieństwa Toksyczności:")
for label, probability in zip(labels_list, probabilities[0]):
    print(f"  {label}: {probability:.4f}")

## 8. Analiza Wyjaśnianości z Integrated Gradients

In [ ]:
from captum.attr import IntegratedGradients

# Define prediction function wrapper for Captum
def predict_function(inputs_embeds, attention_mask=None):
    """Wrapper function that returns model logits for Captum."""
    output = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
    return output.logits

# Initialize Integrated Gradients
integrated_gradients = IntegratedGradients(predict_function)

# Select target label for attribution
# 0=toxic, 1=severe_toxic, 2=obscene, 3=threat, 4=insult, 5=identity_hate
target_label_index = 0
target_name = labels_list[target_label_index]

# Prepare input embeddings
input_ids = inputs.input_ids
input_embeddings = model.distilbert.embeddings(input_ids)

# Create baseline (padding tokens as reference)
reference_input_ids = torch.tensor(
    [tokenizer.pad_token_id] * input_ids.size(1), 
    device=device
).unsqueeze(0)
reference_embeddings = model.distilbert.embeddings(reference_input_ids)

# Prepare attention mask
attention_mask = inputs.attention_mask

# Compute attributions
print(f"Computing attributions for: {target_name}...")

attributions, delta = integrated_gradients.attribute(
    inputs=input_embeddings,
    baselines=reference_embeddings,
    target=target_label_index,
    additional_forward_args=(attention_mask,),
    return_convergence_delta=True
)

print(f"Attribution complete. Convergence delta: {delta.item():.6f}")

In [ ]:
from captum.attr import visualization

# Process attributions for visualization
attributions_sum = attributions.sum(dim=-1).squeeze(0)
attributions_sum = attributions_sum / torch.norm(attributions_sum)
attributions_numpy = attributions_sum.cpu().detach().numpy()

# Get probability for target label
probability_score = probabilities[0][target_label_index].item()
predicted_class_label = "True" if probability_score > 0.5 else "False"

# Convert input IDs to tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

# Create visualization data record
visualization_data = visualization.VisualizationDataRecord(
    word_attributions=attributions_numpy,
    pred_prob=probability_score,
    pred_class=predicted_class_label,
    true_class=1,  # Assume text is toxic
    attr_class=target_name,
    attr_score=attributions_numpy.sum(),
    raw_input_ids=tokens,
    convergence_score=delta
)

print(f"\nLabel explanation: {target_name}")
visualization.visualize_text([visualization_data])

## 9. Analiza Warstwowa Metodą Sondowania

In [ ]:
import numpy as np
from tqdm import tqdm

# Enable hidden states output in model configuration
model.config.output_hidden_states = True

def extract_hidden_states(data_subset, layer_index=4):
    """
    Extracts hidden state representations from a specific layer.
    
    Args:
        data_subset: Dataset subset to extract from
        layer_index: Which transformer layer to extract (0-6 for DistilBERT)
        
    Returns:
        Tuple of (hidden_states_array, labels_array)
    """
    model.eval()
    all_hidden_states = []
    all_labels = []
    
    print(f"Extracting representations from layer {layer_index}...")
    
    for i in tqdm(range(len(data_subset))):
        entry = data_subset[i]
        
        text_tensor = entry['input_ids'].unsqueeze(0).to(device)
        mask_tensor = entry['attention_mask'].unsqueeze(0).to(device)
        label = entry['labels'][0].item()  # Extract first label (toxic)
        
        with torch.no_grad():
            outputs = model(text_tensor, attention_mask=mask_tensor)
            hidden_state = outputs.hidden_states[layer_index]
            # Extract CLS token embedding (first token)
            cls_embedding = hidden_state[0, 0, :].cpu().numpy()
            
            all_hidden_states.append(cls_embedding)
            all_labels.append(label)
    
    return np.array(all_hidden_states), np.array(all_labels)

# Determine subset size for analysis
total_evaluation_samples = len(evaluation_dataset)
target_size = 500
subset_size = min(target_size, total_evaluation_samples)

print(f"Available samples: {total_evaluation_samples}. Using: {subset_size}")

test_subset = evaluation_dataset.select(range(subset_size))

# Extract hidden states from layer 4
hidden_states_matrix, labels_array = extract_hidden_states(test_subset, layer_index=4)

print(f"\nHidden states shape: {hidden_states_matrix.shape}")
print(f"Labels shape: {labels_array.shape}")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Split extracted representations into train/test sets for probe
X_train_probe, X_test_probe, y_train_probe, y_test_probe = train_test_split(
    hidden_states_matrix, labels_array, test_size=0.2, random_state=42
)

# Train linear probe classifier
probe_classifier = LogisticRegression(max_iter=1000)
probe_classifier.fit(X_train_probe, y_train_probe)

# Evaluate probe performance
y_predictions_probe = probe_classifier.predict(X_test_probe)

accuracy = accuracy_score(y_test_probe, y_predictions_probe)
f1 = f1_score(y_test_probe, y_predictions_probe)

print("=== Probe Results (Layer 4) ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Interpretation
if accuracy > 0.80:
    print("\n✓ Layer 4 has strong toxicity representation")
else:
    print("\n✗ Layer 4 does not have strong toxicity representation")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Extract CAV (Concept Activation Vector) from trained probe
concept_vector = probe_classifier.coef_[0]
intercept = probe_classifier.intercept_[0]

# Project test data onto the toxicity concept vector
projected_scores = np.dot(X_test_probe, concept_vector) + intercept

# Separate scores by true label (toxic vs non-toxic)
scores_toxic = projected_scores[y_test_probe == 1]
scores_safe = projected_scores[y_test_probe == 0]

# Create histogram visualization
plt.figure(figsize=(10, 6))

sns.histplot(scores_safe, color="green", label="Non-Toxic", kde=True, alpha=0.5)
sns.histplot(scores_toxic, color="red", label="Toxic", kde=True, alpha=0.5)

plt.axvline(0, color='black', linestyle='--', label="Decision Boundary")
plt.title(f"Activation Distribution Along CAV (Layer 4)\nAccuracy: {accuracy:.2f}, F1: {f1:.2f}")
plt.xlabel("Projection Score (higher = more toxic)")
plt.ylabel("Number of Examples")
plt.legend()
plt.grid(True, alpha=0.3)

plt.show()

## 10. Porównanie Metod XAI i Ewaluacja Wierności

In [ ]:
import quantus
import numpy as np

def model_predict_numpy(model, inputs, **kwargs):
    """
    Prediction function wrapper for Quantus (accepts numpy arrays).
    
    Args:
        model: PyTorch model
        inputs: Numpy array of token IDs [batch_size, seq_len]
        
    Returns:
        Numpy array of probabilities
    """
    model.eval()
    input_tensor = torch.tensor(inputs, device=device).long()
    
    with torch.no_grad():
        outputs = model(input_tensor)
        return torch.sigmoid(outputs.logits).cpu().numpy()

def explain_function_numpy(model, inputs, targets, **kwargs):
    """
    Explanation function wrapper for Quantus using Integrated Gradients.
    
    Args:
        model: PyTorch model
        inputs: Numpy array of token IDs
        targets: Array of target class indices
        
    Returns:
        Numpy array of attribution scores per token
    """
    model.eval()
    input_tensor = torch.tensor(inputs, device=device).long()
    
    # Create embeddings
    input_embeddings = model.distilbert.embeddings(input_tensor)
    
    # Create baseline (padding)
    reference_input_ids = torch.tensor(
        [tokenizer.pad_token_id] * inputs.shape[1], 
        device=device
    ).unsqueeze(0)
    reference_embeddings = model.distilbert.embeddings(reference_input_ids)
    
    # Initialize Integrated Gradients
    integrated_gradients = IntegratedGradients(lambda x: model(inputs_embeds=x).logits)
    
    # Process each example in batch
    attributions_list = []
    for i in range(len(inputs)):
        target_index = int(targets[i])
        
        attribution = integrated_gradients.attribute(
            inputs=input_embeddings[i].unsqueeze(0),
            baselines=reference_embeddings,
            target=target_index,
            n_steps=20  # Fewer steps for faster computation
        )
        
        # Sum over embedding dimension to get per-token importance
        attribution_sum = attribution.sum(dim=-1).squeeze(0).cpu().detach().numpy()
        attributions_list.append(attribution_sum)
    
    return np.array(attributions_list)

In [ ]:
import torch
import numpy as np

# Find toxic examples in dataset
toxic_indices = np.where(labels_array == 1)[0]

# Select batch of examples for evaluation
batch_size = 16
selected_indices = toxic_indices[:batch_size]

# Extract input IDs for selected examples
input_batch_toxic = [test_subset[int(i)]['input_ids'] for i in selected_indices]
target_batch = labels_array[selected_indices]

print(f"Selected {len(input_batch_toxic)} toxic examples for evaluation.")

# Configuration for faithfulness test
top_k_tokens = 5  # Number of most important tokens to remove
dataset_samples = input_batch_toxic
targets = target_batch

print(f"\n=== Faithfulness Evaluation (Comprehensiveness) ===")
print(f"Testing on {len(dataset_samples)} examples")
print(f"Removing {top_k_tokens} most important tokens from each sentence\n")

comprehensiveness_scores = []

# Evaluate each example
for i in range(len(dataset_samples)):
    # Prepare single input
    input_id = dataset_samples[i].unsqueeze(0).to(device)
    
    # Get original prediction
    model.eval()
    with torch.no_grad():
        original_output = model(input_id)
        original_probability = torch.sigmoid(original_output.logits)[0][0].item()
    
    # Compute attributions using Integrated Gradients
    integrated_gradients = IntegratedGradients(predict_function)
    
    # Prepare embeddings
    input_embedding = model.distilbert.embeddings(input_id)
    baseline_embedding = model.distilbert.embeddings(
        torch.tensor([tokenizer.pad_token_id] * input_id.size(1), device=device).unsqueeze(0)
    )
    
    # Compute attributions
    attribution, _ = integrated_gradients.attribute(
        inputs=input_embedding,
        baselines=baseline_embedding,
        target=0,  # Targeting toxic class
        return_convergence_delta=True
    )
    
    # Sum attributions to token level
    attribution_sum = attribution.sum(dim=-1).squeeze(0)
    
    # Find top-K most important tokens
    _, top_indices = torch.topk(attribution_sum, k=top_k_tokens)
    
    # Perturb input by masking important tokens
    perturbed_input_id = input_id.clone()
    perturbed_input_id[0, top_indices] = tokenizer.pad_token_id
    
    # Get prediction on perturbed input
    with torch.no_grad():
        perturbed_output = model(perturbed_input_id)
        perturbed_probability = torch.sigmoid(perturbed_output.logits)[0][0].item()
    
    # Compute comprehensiveness score (confidence drop)
    confidence_drop = original_probability - perturbed_probability
    comprehensiveness_scores.append(confidence_drop)
    
    # Display first example details
    if i == 0:
        print(f"Example 1 - Original confidence: {original_probability:.4f}")
        print(f"Example 1 - After removing top-{top_k_tokens} tokens: {perturbed_probability:.4f}")
        print(f"Example 1 - Confidence drop: {confidence_drop:.4f}")
        removed_words = tokenizer.convert_ids_to_tokens(input_id[0, top_indices])
        print(f"Removed tokens: {removed_words}\n")

# Display final results
average_score = np.mean(comprehensiveness_scores)
std_score = np.std(comprehensiveness_scores)

print("=" * 50)
print(f"Average Comprehensiveness Score: {average_score:.4f}")
print(f"Standard Deviation: {std_score:.4f}")

if average_score > 0.1:
    print("\n✅ Integrated Gradients works well!")
    print("   Removing identified tokens significantly reduces toxicity.")
else:
    print("\n❌ Integrated Gradients poorly identifies important tokens.")
    print("   Model still detects toxicity after perturbation.")

In [ ]:
from captum.attr import InputXGradient

# Initialize Input × Gradient method
input_x_gradient = InputXGradient(predict_function)

print(f"Computing attributions using Input×Gradient for: {target_name}...")

# Compute attributions (no baseline needed for Input×Gradient)
attributions_ixg = input_x_gradient.attribute(
    inputs=input_embeddings,
    target=target_label_index,
    additional_forward_args=(attention_mask,)
)

# Process results for visualization
attributions_ixg_sum = attributions_ixg.sum(dim=-1).squeeze(0)
attributions_ixg_sum = attributions_ixg_sum / torch.norm(attributions_ixg_sum)
attributions_ixg_numpy = attributions_ixg_sum.cpu().detach().numpy()

# Create visualization data record for Input×Gradient
visualization_data_ixg = visualization.VisualizationDataRecord(
    word_attributions=attributions_ixg_numpy,
    pred_prob=probability_score,
    pred_class=predicted_class_label,
    true_class=1,
    attr_class=f"{target_name} (Input×Gradient)",
    attr_score=attributions_ixg_numpy.sum(),
    raw_input_ids=tokens,
    convergence_score=None  # Input×Gradient doesn't compute convergence delta
)

print("\n=== XAI Methods Comparison ===")
print("Row 1: Integrated Gradients")
print("Row 2: Input × Gradient")

# Visualize both methods side by side
visualization.visualize_text([visualization_data, visualization_data_ixg])

## 11. Kompleksowa Analiza Wszystkich Warstw

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

def extract_all_layers(dataset, model, device, batch_size=32):
    """
    Efficiently extracts CLS embeddings from all transformer layers in a single pass.
    
    Args:
        dataset: HuggingFace dataset to extract from
        model: Transformer model with output_hidden_states enabled
        device: PyTorch device (cuda/cpu)
        batch_size: Batch size for processing
        
    Returns:
        Tuple of (layers_dict, labels_array)
        - layers_dict: Dictionary mapping layer_index -> numpy array of embeddings
        - labels_array: Numpy array of labels
    """
    model.eval()
    
    # Create DataLoader for efficient batching
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # Dictionary to store activations per layer
    # DistilBERT has: 1 embedding layer + 6 transformer layers = 7 hidden states
    layers_data = {}
    all_labels = []
    
    print(f"Extracting from {len(dataset)} samples...")
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Layer Extraction"):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']
            
            # Single forward pass retrieves all hidden states
            outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            
            # Extract CLS token (index 0) from each layer
            for layer_index, hidden_state in enumerate(outputs.hidden_states):
                if layer_index not in layers_data:
                    layers_data[layer_index] = []
                
                # Extract CLS embeddings [batch_size, hidden_dim]
                cls_embeddings = hidden_state[:, 0, :].cpu().numpy()
                layers_data[layer_index].append(cls_embeddings)
            
            # Extract toxic label (first column)
            if labels.dim() > 1:
                toxic_labels = labels[:, 0].cpu().numpy()
            else:
                toxic_labels = labels.cpu().numpy()
            
            all_labels.extend(toxic_labels)
    
    # Concatenate all batches
    final_layer_activations = {
        layer: np.concatenate(data, axis=0)
        for layer, data in layers_data.items()
    }
    final_labels = np.array(all_labels)
    
    return final_layer_activations, final_labels

# Determine analysis subset size
evaluation_subset_size = 1000
if len(evaluation_dataset) > evaluation_subset_size:
    analysis_dataset = evaluation_dataset.select(range(evaluation_subset_size))
else:
    analysis_dataset = evaluation_dataset

# Extract representations from all layers
layers_activations_dict, all_labels = extract_all_layers(
    analysis_dataset, model, device, batch_size=32
)

print(f"\nExtraction complete. Layers extracted: {list(layers_activations_dict.keys())}")
print(f"Layer 0 shape: {layers_activations_dict[0].shape}")

# Train probes for each layer
probe_results = []

print("\nTraining linear probes for each layer...")

for layer_index in sorted(layers_activations_dict.keys()):
    X = layers_activations_dict[layer_index]
    y = all_labels
    
    # Binarize labels
    y = (y > 0.5).astype(int)
    
    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train logistic regression probe
    classifier = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
    classifier.fit(X_train, y_train)
    
    # Evaluate
    y_pred = classifier.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    probe_results.append({
        'layer': layer_index,
        'accuracy': acc,
        'f1': f1
    })
    
    print(f"Layer {layer_index}: Accuracy={acc:.4f}, F1={f1:.4f}")

# Convert results to DataFrame
results_dataframe = pd.DataFrame(probe_results)

# Visualize results
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Plot lines for accuracy and F1
sns.lineplot(data=results_dataframe, x='layer', y='accuracy', 
             marker='o', label='Accuracy', linewidth=2.5)
sns.lineplot(data=results_dataframe, x='layer', y='f1', 
             marker='s', label='F1 Score', linewidth=2.5)

# Format plot
plt.title("Linear Separability of Toxicity Across Layers (DistilBERT)", fontsize=14, pad=15)
plt.xlabel("Layer Number (0=Embeddings, 1-6=Transformer Layers)", fontsize=12)
plt.ylabel("Metric Value", fontsize=12)
plt.ylim(0.0, 1.05)
plt.xticks(results_dataframe['layer'])
plt.legend(fontsize=11)

# Add value labels above points
for index, row in results_dataframe.iterrows():
    plt.text(row['layer'], row['accuracy'] + 0.01, f"{row['accuracy']:.2f}",
             ha='center', color='blue', fontsize=9)

plt.tight_layout()
plt.show()

## 12. Analiza Stabilności z Generowaniem Parafraz

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from captum.attr import IntegratedGradients

# Load T5 paraphrase generation model
print("Loading T5 paraphrase model...")
paraphrase_model_name = "Vamsi/T5_Paraphrase_Paws"
paraphrase_tokenizer = AutoTokenizer.from_pretrained(paraphrase_model_name)
paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained(paraphrase_model_name).to(device)
print("T5 model loaded!")

def generate_paraphrase(text, num_return_sequences=1):
    """
    Generates paraphrase for given text using T5 model.
    
    Args:
        text: Input text to paraphrase
        num_return_sequences: Number of paraphrases to generate
        
    Returns:
        Paraphrased text string
    """
    paraphrase_model.eval()
    
    # T5 requires task prefix for this specific model
    text = "paraphrase: " + text + " </s>"
    
    encoding = paraphrase_tokenizer.encode_plus(
        text,
        padding="longest",
        return_tensors="pt"
    )
    
    input_ids = encoding["input_ids"].to(device)
    attention_masks = encoding["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = paraphrase_model.generate(
            input_ids=input_ids,
            attention_mask=attention_masks,
            max_length=256,
            do_sample=True,  # Sampling allows greater diversity
            top_k=120,
            top_p=0.95,
            early_stopping=True,
            num_return_sequences=num_return_sequences
        )
    
    # Decode output
    paraphrase = paraphrase_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return paraphrase

def get_top_k_tokens(text_input, model, tokenizer, k=5):
    """
    Computes Integrated Gradients attributions and returns set of k most important tokens.
    
    Args:
        text_input: Input text to analyze
        model: Classification model
        tokenizer: Tokenizer for the model
        k: Number of top tokens to return
        
    Returns:
        Set of most important token strings
    """
    # Prepare input for DistilBERT
    inputs = tokenizer(text_input, return_tensors="pt", truncation=True, padding=True).to(device)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    
    # Define prediction function for IG
    def predict_func(inputs_embeds):
        out = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
        return out.logits
    
    integrated_gradients = IntegratedGradients(predict_func)
    
    # Prepare embeddings
    input_embeddings = model.distilbert.embeddings(input_ids)
    reference_input_ids = torch.tensor(
        [tokenizer.pad_token_id] * input_ids.size(1), 
        device=device
    ).unsqueeze(0)
    reference_embeddings = model.distilbert.embeddings(reference_input_ids)
    
    # Compute attributions (target=0 for 'toxic' class)
    target_index = 0
    
    attributions, _ = integrated_gradients.attribute(
        inputs=input_embeddings,
        baselines=reference_embeddings,
        target=target_index,
        return_convergence_delta=True
    )
    
    # Sum and select top-K
    attribution_sum = attributions.sum(dim=-1).squeeze(0)
    _, top_indices = torch.topk(attribution_sum, k=min(k, len(attribution_sum)))
    
    # Convert IDs to tokens
    top_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][top_indices])
    
    # Clean tokens (remove '##' from subwords and convert to lowercase)
    clean_tokens = set([
        token.replace("##", "").lower() 
        for token in top_tokens 
        if token not in ['[CLS]', '[SEP]', '[PAD]']
    ])
    
    return clean_tokens

def evaluate_stability(original_text, layer_index, model, tokenizer):
    """
    Computes three stability metrics:
    1. Output Stability - How much prediction changes
    2. Layer Stability - Cosine similarity of representations
    3. Attribution Stability - Jaccard similarity of important tokens
    
    Args:
        original_text: Original input text
        layer_index: Which layer to analyze
        model: Classification model
        tokenizer: Tokenizer for the model
        
    Returns:
        Dictionary with stability metrics
    """
    # Generate paraphrase
    paraphrase_text = generate_paraphrase(original_text)
    
    # Prepare both texts
    inputs_original = tokenizer(
        original_text, return_tensors="pt", truncation=True, max_length=512
    ).to(device)
    inputs_paraphrase = tokenizer(
        paraphrase_text, return_tensors="pt", truncation=True, max_length=512
    ).to(device)
    
    model.eval()
    
    # Run model with hidden states output
    with torch.no_grad():
        output_original = model(**inputs_original, output_hidden_states=True)
        output_paraphrase = model(**inputs_paraphrase, output_hidden_states=True)
    
    # A. Output Stability (prediction difference)
    probability_original = torch.sigmoid(output_original.logits)[0][0].item()
    probability_paraphrase = torch.sigmoid(output_paraphrase.logits)[0][0].item()
    prediction_difference = abs(probability_original - probability_paraphrase)
    
    # B. Layer Stability (cosine similarity of CLS representations)
    cls_original = output_original.hidden_states[layer_index][:, 0, :]  # [1, 768]
    cls_paraphrase = output_paraphrase.hidden_states[layer_index][:, 0, :]  # [1, 768]
    
    cosine_similarity = F.cosine_similarity(cls_original, cls_paraphrase).item()
    
    # C. Attribution Stability (Jaccard Index of top tokens)
    tokens_original = get_top_k_tokens(original_text, model, tokenizer, k=5)
    tokens_paraphrase = get_top_k_tokens(paraphrase_text, model, tokenizer, k=5)
    
    # Compute Jaccard Index
    intersection = len(tokens_original.intersection(tokens_paraphrase))
    union = len(tokens_original.union(tokens_paraphrase))
    jaccard_score = intersection / union if union > 0 else 0.0
    
    return {
        "Original Text": original_text,
        "Paraphrase": paraphrase_text,
        "Prob Original": round(probability_original, 4),
        "Prob Paraphrase": round(probability_paraphrase, 4),
        "Pred Diff (Output)": round(prediction_difference, 4),
        "Layer Cosine Sim": round(cosine_similarity, 4),
        "Attribution Jaccard": round(jaccard_score, 4),
        "Top Tokens Orig": list(tokens_original),
        "Top Tokens Para": list(tokens_paraphrase)
    }

In [ ]:
# Find toxic examples for stability testing
toxic_indices = [i for i, x in enumerate(y_test_probe) if x == 1][:15]
if len(toxic_indices) == 0:
    print("No toxic samples found, using random selection...")
    toxic_indices = range(10)

print(f"\nStarting stability analysis for {len(toxic_indices)} examples...")
print(f"Analyzing layer: 5 (based on previous analysis results)")

stability_results = []

# Evaluate stability for each example
for idx in toxic_indices:
    # Decode tokenized text
    input_ids_raw = test_subset[idx]['input_ids']
    original_text = tokenizer.decode(input_ids_raw, skip_special_tokens=True)
    
    # Run stability evaluation on Layer 5
    metrics = evaluate_stability(original_text, layer_index=5, model=model, tokenizer=tokenizer)
    stability_results.append(metrics)

# Create DataFrame with results
stability_dataframe = pd.DataFrame(stability_results)

# Display results
pd.set_option('display.max_colwidth', 50)
display(stability_dataframe[[
    "Original Text", "Paraphrase",
    "Pred Diff (Output)", "Layer Cosine Sim", "Attribution Jaccard"
]])

# Summary statistics
print("\n=== Stability Summary (Averages) ===")
print(f"Mean Prediction Stability (Diff): {stability_dataframe['Pred Diff (Output)'].mean():.4f}")
print(f"  (Lower is better - less variation in predictions)")
print(f"Mean Layer Stability (Cosine):    {stability_dataframe['Layer Cosine Sim'].mean():.4f}")
print(f"  (Higher is better - closer to 1.0 indicates more stable representations)")
print(f"Mean Attribution Stability (Jacc): {stability_dataframe['Attribution Jaccard'].mean():.4f}")
print(f"  (Higher is better - closer to 1.0 indicates consistent explanations)")

## 13. Inżynieria Reprezentacji - Budowanie Wektorów Sterujących

In [ ]:
# Extract steering vector using Difference of Means method

# Separate Layer 5 activations by toxic/safe labels
layer_5_activations = layers_activations_dict[5]
is_toxic = (all_labels > 0.5)  # Boolean mask for toxic examples

# Compute centroids for both groups
mean_toxic = np.mean(layer_5_activations[is_toxic], axis=0)
mean_safe = np.mean(layer_5_activations[~is_toxic], axis=0)

# Compute direction vector (from safe to toxic)
direction_vector = mean_toxic - mean_safe

# Analyze vector properties for debugging
vector_norm = np.linalg.norm(direction_vector)
hidden_state_norm = np.linalg.norm(mean_safe)

print(f"Steering vector norm (Difference of Means): {vector_norm:.4f}")
print(f"Average activation norm in model: {hidden_state_norm:.4f}")
print(f"Ratio: {vector_norm / hidden_state_norm:.4f}")

# Convert to PyTorch tensor (preserve natural scale)
steering_tensor = torch.tensor(direction_vector, dtype=torch.float32).to(device)

print("\nSteering vector (Mean Difference) ready!")

# Determine optimal alpha scaling based on vector norm
scale_factor = 5.0
suggested_alpha = scale_factor

print(f"\nSuggested alpha strength: +/- {suggested_alpha}")

In [ ]:
def predict_with_steering(text, model, tokenizer, steering_vector, alpha=0):
    """
    Makes prediction with optional steering vector intervention.
    
    Args:
        text: Input text to classify
        model: Classification model
        tokenizer: Tokenizer for the model
        steering_vector: Direction vector for intervention
        alpha: Steering strength (negative detoxifies, positive amplifies)
        
    Returns:
        Probability of toxic class
    """
    model.eval()
    
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
    
    # Define hook for intervention
    class SteeringHook:
        def __init__(self, vector, coefficient):
            self.vector = vector
            self.coefficient = coefficient
        
        def __call__(self, module, inputs, output):
            hidden_states = output[0]
            steered_states = hidden_states + (self.coefficient * self.vector)
            return (steered_states,) + output[1:]
    
    # Register hook on Layer 5 if alpha != 0
    if alpha != 0:
        hook_handle = model.distilbert.transformer.layer[5].register_forward_hook(
            SteeringHook(steering_vector, alpha)
        )
    
    # Run prediction
    with torch.no_grad():
        outputs = model(**inputs)
        probability = torch.sigmoid(outputs.logits)[0][0].item()
    
    # Remove hook
    if alpha != 0:
        hook_handle.remove()
    
    return probability

# Test steering on toxic sentence
test_toxic = "You are a complete idiot and a waste of time."

score_original = predict_with_steering(test_toxic, model, tokenizer, steering_tensor, alpha=0)
score_detoxified = predict_with_steering(test_toxic, model, tokenizer, steering_tensor, alpha=-suggested_alpha)
score_amplified = predict_with_steering(test_toxic, model, tokenizer, steering_tensor, alpha=suggested_alpha)

print(f"\nSentence: {test_toxic}")
print(f"Original (Alpha 0):       {score_original:.4f}")
print(f"Detoxified (Alpha -{suggested_alpha}): {score_detoxified:.4f} (expect decrease)")
print(f"Amplified (Alpha +{suggested_alpha}):  {score_amplified:.4f} (expect increase)")

# Plot steering effects across alpha range
alphas = np.linspace(-suggested_alpha * 2, suggested_alpha * 2, 10)
scores = [predict_with_steering(test_toxic, model, tokenizer, steering_tensor, alpha=a) for a in alphas]

plt.figure(figsize=(8, 5))
plt.plot(alphas, scores, marker='o', color='green', linewidth=2)
plt.axhline(0.5, color='gray', linestyle='--', label='Classification Threshold')
plt.title(f"Mean Difference Steering (Layer 5)\nSentence: '{test_toxic[:30]}...'")
plt.xlabel("Alpha (Negative = Detoxification)")
plt.ylabel("Toxic Probability")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Demonstrate classification flip
print("\n=== Classification Change Demo ===")
base_score = predict_with_steering(test_toxic, model, tokenizer, steering_tensor, alpha=0)
detox_score = predict_with_steering(test_toxic, model, tokenizer, steering_tensor, alpha=-15)

print(f"Sentence: {test_toxic}")
print(f"Original:  {base_score:.4f} (Toxic? {base_score > 0.5})")
print(f"Steered:   {detox_score:.4f} (Toxic? {detox_score > 0.5})")

## 14. Testowanie Efektów Ubocznych i Ocena Bezpieczeństwa

In [ ]:
# Set optimal alpha based on steering experiments
optimal_alpha = -3.0

# Test sentences from different categories
test_sentences = [
    # Toxic sentences (should be detoxified)
    ("You are a complete idiot and a waste of time.", "Toxic"),
    ("I hope you fail miserably, you loser.", "Toxic"),
    
    # Safe positive sentences (should remain unchanged)
    ("I really love the weather today, it is beautiful.", "Safe_Positive"),
    ("Thank you for your help, I appreciate it.", "Safe_Positive"),
    
    # Safe negative sentences (important edge case - should not be flagged)
    ("I am feeling very sad and tired today.", "Safe_Negative"),
    ("The movie was boring and too long.", "Safe_Negative"),
    ("I disagree with your opinion regarding the tax policy.", "Safe_Negative")
]

print(f"=== Side Effects Test (Alpha = {optimal_alpha}) ===\n")
print(f"{'Category':<20} | {'Original':<12} | {'Steered':<12} | {'Status'}")
print("=" * 70)

for text, category in test_sentences:
    # Prediction without intervention
    probability_original = predict_with_steering(text, model, tokenizer, steering_tensor, alpha=0)
    
    # Prediction with detoxification
    probability_steered = predict_with_steering(text, model, tokenizer, steering_tensor, alpha=optimal_alpha)
    
    # Evaluate results
    if category == "Toxic":
        # For toxic: success if drops below 0.5
        if probability_steered < 0.1:
            status = "✅ Fixed"
        elif probability_steered < 0.5:
            status = "⚠️  Improved"
        else:
            status = "❌ Failed"
    else:
        # For safe: check that it doesn't break (shouldn't become toxic)
        change = abs(probability_original - probability_steered)
        if probability_steered > 0.5:
            status = "❌ BROKEN (False Positive)"
        elif change < 0.2:
            status = "✅ Stable"
        else:
            status = "⚠️  Shifted"
    
    print(f"{category:<20} | {probability_original:.4f}       | {probability_steered:.4f}       | {status}")

## 15. Wdrożenie Produkcyjne - Zapisywanie i Ładowanie Artefaktów Sterowania

In [ ]:
import torch
from datetime import datetime

# Create steering artifact for production use
steering_artifact = {
    "steering_vector": steering_tensor.cpu(),  # Move to CPU for saving
    "layer_index": 5,                          # Target layer
    "alpha": optimal_alpha,                    # Optimal steering strength (-3.0)
    "method": "mean_difference",
    "model_name": "distilbert-base-uncased",
    "description": "Vector for removing toxicity concept from layer 5"
}

# Save with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
save_path = f"/drive/MyDrive/msc-project/vectors/toxicity_steering_controller_{timestamp}.pt"

torch.save(steering_artifact, save_path)
print(f"✅ Steering artifact saved to: {save_path}")

# ============================================================
# PRODUCTION SIMULATION - Clean session without training data
# ============================================================

print("\n=== Production Environment Simulation ===")

# Load artifact
artifact = torch.load(save_path)
loaded_vector = artifact["steering_vector"].to(device)
loaded_layer = artifact["layer_index"]
loaded_alpha = artifact["alpha"]

print(f"Loaded controller: {artifact['description']}")
print(f"Configuration: Layer {loaded_layer}, Alpha {loaded_alpha}")

# Define production hook class
class ProductionSteeringHook:
    """Hook for applying steering vector in production inference."""
    def __init__(self, vector, coefficient):
        self.vector = vector
        self.coefficient = coefficient
    
    def __call__(self, module, inputs, output):
        hidden_states = output[0]
        steered_states = hidden_states + (self.coefficient * self.vector)
        return (steered_states,) + output[1:]

# Production inference function
def generate_safe_prediction(text, model, tokenizer):
    """
    Run inference with built-in detoxification.
    
    Args:
        text: Input text to classify
        model: Classification model
        tokenizer: Tokenizer for the model
        
    Returns:
        Toxic probability after steering
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
    
    # Register steering hook
    hook = model.distilbert.transformer.layer[loaded_layer].register_forward_hook(
        ProductionSteeringHook(loaded_vector, loaded_alpha)
    )
    
    # Run prediction
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probability = torch.sigmoid(outputs.logits)[0][0].item()
    
    # Clean up hook
    hook.remove()
    
    return probability

# Live test
live_test_text = "You are completely useless and stupid."
safety_score = generate_safe_prediction(live_test_text, model, tokenizer)

print(f"\nLive Test Input: '{live_test_text}'")
print(f"Model Toxic Probability (Steered): {safety_score:.4f}")
print(f"Decision: {'🔴 BLOCK' if safety_score > 0.5 else '🟢 ALLOW'}")

## Podsumowanie Eksperymentu: Inżynieria Reprezentacji dla Detoksykacji Modelu

Ten projekt demonstruje kompleksową analizę i modyfikację wewnętrznych reprezentacji w modelu DistilBERT (wytrenowanym na danych Jigsaw Toxicity) w celu kontrolowania jego zachowania bez ponownego treningu.

### Zrealizowane Etapy:

#### 1. Analiza Warstwowa
- Zbadano liniową separowalność koncepcji "toksyczność" w całej sieci neuronowej
- Zidentyfikowano **Warstwę 5** jako optymalny punkt interwencji (F1 Score = 0.80)
- Ta warstwa przewyższyła warstwę końcową pod względem jakości reprezentacji, wskazując "sweet spot" dla rozumienia semantycznego

#### 2. Analiza Stabilności
- Użyto generatora parafraz T5 do testowania robustności reprezentacji
- Wykazano, że aktywacje Warstwy 5 są niezwykle stabilne semantycznie (Podobieństwo Cosine > 0.99)
- Potwierdzono, że reprezentacje pozostają spójne nawet gdy struktura zdania się zmienia, walidując wykonalność interwencji

#### 3. Ekstrakcja Wektora Sterującego
- Zastosowano metodę **Difference of Means** do obliczenia wektora kierunkowego między centroidami aktywacji toksycznych i bezpiecznych w Warstwie 5
- To podejście okazało się bardziej skuteczne niż wagi regresji logistycznej, zapewniając odpowiednie skalowanie sygnału
- Pomyślnie przechwycono kierunek koncepcji toksyczności w przestrzeni reprezentacji

#### 4. Sterowanie Modelem i Interwencja
- Zaimplementowano mechanizm PyTorch Forward Hook dla wstrzykiwania wektora sterującego w czasie rzeczywistym
- Zastosowano interwencję z siłą Alpha = -3.0 dla efektywnej "detoksykacji" modelu
- Osiągnięto kontrolowaną redukcję detekcji toksyczności bez ponownego treningu modelu

#### 5. Ewaluacja i Zapewnienie Jakości

**Skuteczność:**
- Prawdopodobieństwo toksyczności dla obraźliwych fraz spadło z ~92% do ~1-4%

**Bezpieczeństwo:**
- Model zachował poprawne zachowanie dla zdań neutralnych i pozytywnych (brak efektu "lobotomii")

**Redukcja Fałszywych Pozytywów:**
- Wyeliminowano niepoprawne oznaczanie zdań o negatywnym sentymencie (np. skargi) jako toksyczne
- Wskaźnik fałszywych pozytywów spadł z 10% do 0%

### Wnioski

Ten projekt potwierdza, że **Inżynieria Reprezentacji (RepE)** jest potężną, niskokosztową metodą kontrolowania zachowania modeli LLM/BERT. Poprzez precyzyjne operacje na wektorach aktywacji w Warstwie 5, pomyślnie wyeliminowaliśmy niepożądane zachowanie modelu (detekcję toksyczności) zachowując jego ogólne możliwości językowe.

Podejście demonstruje, że manipulacja wewnętrznymi reprezentacjami oferuje realną alternatywę dla kosztownego ponownego treningu, umożliwiając precyzyjną kontrolę nad wyjściami modelu poprzez celowane interwencje w przestrzeni aktywacji.
